In [76]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from tqdm import tqdm
import pickle

In [82]:
def approx_dist(lat: float, lon: float) -> float:
    dx = lat * 110.574 # km
    dy = lon * 111.320 * np.cos(lat) # km
    return np.sqrt(dx**2 + dy**2)

In [77]:
# Loading data
recensement = pd.read_excel(io="../../1832_v4.xlsx", na_values=["·","?"])
classes = pd.read_excel(io="../../histoire urbaine digitale classification metiers.xlsx", sheet_name="data",  na_values="·")
rues = pd.read_csv("../../nom_rues_et_coor.csv", sep=",")

# Adding the 'chef_vocation_categorie' column
classes = classes.rename(columns={"JOB": "chef_vocation_norm_2"})
classes = classes.rename(columns={"CLASS": "chef_vocation_categorie"})
recensement = recensement.merge(classes, on="chef_vocation_norm_2", how="outer")

# Split the values on the '/' character
split_vals = recensement['chef_vocation_categorie'].str.split('/')

# Create a new dataframe from the split values
recensement = recensement.assign(chef_vocation_categorie=split_vals).explode('chef_vocation_categorie')

# Adding coordinates
correspondances = {
    "nom_rue": [
        "ale", 
        "etraz", 
        "rue du pre", 
        "chaucrau", 
        "st laurent", 
        "st pierre", 
        "st etienne", 
        "bourg", 
        "st francois", 
        "georgette",
    ],
    "nom_rue_norm_2": [
        "rue de l'ale",
        "rue d etraz",
        "rue du pre",
        "rue de chaucrau",
        "place de st laurent",
        "rue de st pierre",
        "st etienne",
        "rue de bourg",
        "place de st froncois",
        "chemin de georgette",
    ]
}
for index, row in tqdm(recensement.iterrows(), total=len(recensement), desc="Adding coordinates"):
    nom_rue = row["nom_rue_norm_2"]
    if type(nom_rue) == float: continue # if nom_rue is NaN
    if nom_rue in correspondances["nom_rue"]:
        index = correspondances["nom_rue"].index(nom_rue)
        nom_rue = correspondances["nom_rue_norm_2"][index]
        corresponding_streets = rues.query("Nom_rue == @nom_rue")
    else:
        corresponding_streets = rues.query("Nom_rue.str.contains(@nom_rue)")
    occurences = len(corresponding_streets)
    if occurences == 1:
        recensement.at[index, "rue_x"] = corresponding_streets["X"].values[0]
        recensement.at[index, "rue_y"] = corresponding_streets["Y"].values[0]
        continue
print(f'{len(recensement.query("~rue_x.isnull()")) / len(recensement) * 100:.2f}% des personnes positionnées')

In [ ]:
recensement = recensement.query("not chef_vocation_categorie.isna()")

In [ ]:
G = nx.Graph()

for nom_rue in recensement["nom_rue_norm_2"].unique():
    try:
        G.add_node(
            nom_rue,
            DisplayName=nom_rue.capitalize(),
            DataType="rue",
            vocation_categorie="none",
        )
    except:
        continue

for origine in recensement["chef_origine_norm_2"].unique():
    try:
        G.add_node(
            origine,
            DisplayName=origine.capitalize(),
            DataType="origine",
            vocation_categorie="none",
        )
    except:
        continue

for _, row in recensement.iterrows():
    try:
        nom = row["chef_nom"]
        prenom = row["chef_prenom"]
        vocation_categorie = row["chef_vocation_categorie"]
        internname = f"{nom.upper()} {prenom.capitalize()}"
        # NbEnfants = len(("" if type(row["enfants_dans_la_commune_prenom_norm"]) != str else row["enfants_dans_la_commune_prenom_norm"]).split("|"))
        G.add_node(
            internname,
            DisplayName="",
            DataType="individu",
            vocation_categorie=vocation_categorie,
            # NbEnfants=NbEnfants,
        )
        G.add_edge(
            internname,
            row["nom_rue_norm_2"],
            vocation_categorie=vocation_categorie,
            weight=1,
        )
        G.add_edge(
            internname,
            row["chef_origine_norm_2"],
            vocation_categorie=vocation_categorie,
            weight=1,
        )
    except:
        continue
    
nx.write_gexf(G, "tripartite_individus_origine_rue.gexf")